### Spark Streaming - Contagem de Palavras

#### Arquivo Utilizado: `contar-palavras.txt`
O arquivo contém informações sobre o projeto Apache Spark, e vamos usar seu conteúdo para contar as palavras.

> path: https://raw.githubusercontent.com/rafael-negrao/laboratorio-spark/main/dados/contar-palavras.txt


#### Mover o arquivo do driver e colocar do dbfs

In [0]:
%sql
create volume if not exists `laboratorio-spark`;

In [0]:
dbutils.fs.mkdirs("dbfs:/Volumes/workspace/default/laboratorio-spark/contar-palavras/")

In [0]:
%sh
wget https://raw.githubusercontent.com/rafael-negrao/laboratorio-spark/main/dados/contar-palavras.txt -O /Volumes/workspace/default/laboratorio-spark/contar-palavras/contar-palavras.txt

In [0]:
dbutils.fs.mkdirs("dbfs:/Volumes/workspace/default/laboratorio-spark/contar-palavras/")


In [0]:
print(dbutils.fs.cp(
    "dbfs:/Volumes/workspace/default/laboratorio-spark/contar-palavras/contar-palavras.txt", 
    "dbfs:/Volumes/workspace/default/laboratorio-spark/contar-palavras/contar-palavras1.txt"))
display(dbutils.fs.ls("/Volumes/workspace/default/laboratorio-spark/contar-palavras/"))


#### Criar processo de streaming

Aqui está o código Spark que lê o fluxo de texto de um socket, conta as palavras e exibe o resultado no console:


In [0]:
# Ler o fluxo de dados a partir de um diretório de arquivos de texto
lines = (
   spark.readStream
      .format("text")
      .option("path", "/Volumes/workspace/default/laboratorio-spark/contar-palavras")
      .load()
)


#### Aplicando processo de transformação


In [0]:
import pyspark.sql.functions as F
# Continuar com o restante do código igual
words = (
  lines.select(
    F.explode(F.split(lines.value, " ")).alias("palavra"))
)
word_counts = words.groupBy("palavra").count()


#### Carregando o streaming


In [0]:
# Exibir no console
query = (
   word_counts
      .writeStream
      .outputMode("complete")
      .format("console")
      .start()
)

query.awaitTermination()

In [0]:
# display(word_counts, checkpointLocation="/Volumes/workspace/default/laboratorio-spark/contar-palavras/_checkpoint")
display(word_counts)

`[INFO]: FIM DO NOTEBOOK`